In [ ]:
# Code reference: 
# https://medium.com/@duyanhnguyen_38925/create-a-strong-text-classification-with-the-help-from-elmo-e90809ba29da
# https://www.kaggle.com/saikumar587/text-classification-elmo

import pandas as pd

#import the train and test data
url = 'https://raw.githubusercontent.com/XinyanHe/413Dataset/master/news.csv'
train_news = pd.read_csv(url)

#pick 5000 data from datasets
train_news = train_news.head(5000)
# !
# training data: 4000
# test data: 1000

In [ ]:
#install ELMO
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow.compat.v1 as tf1
import numpy as np

tf1.disable_eager_execution()
tf1.disable_v2_behavior()

#download the elmo model
elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

#create an embedding layer for elmo
def elmo_embedding(x):
    return elmo(tf.squeeze(tf.cast(x,tf.string)),signature='default',as_dict=True)["default"]

In [ ]:
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.regularizers

def build_model(): 
    input_text = Input(shape=(1,), dtype=tf.string)
    embedding = Lambda(elmo_embedding, output_shape=(1024, ))(input_text)
    dense = Dense(256, activation='relu')(embedding)
    pred = Dense(4, activation='softmax')(dense)
    model = Model(inputs=[input_text], outputs=pred)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
def encode(le_enc, labels):
    enc = le_enc.transform(labels)
    return keras.utils.to_categorical(enc)

def decode(le_enc, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le_enc.inverse_transform(dec)

In [ ]:
#build our classification model
model_elmo = build_model()
model_elmo.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               262400    
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 1028      
Total params: 263,428
Trainable params: 263,428
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from sklearn import metrics,preprocessing,model_selection
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical

X = train_news['Description'].tolist()
y = train_news['Class Index'].tolist()

#fitting the model
le = preprocessing.LabelEncoder()
le.fit(y)

y_en = encode(le,y)
x_train, x_test, y_train, y_test = model_selection.train_test_split(np.asarray(X), np.asarray(y_en), test_size=0.2, random_state=42)

In [ ]:
with tf1.Session() as session:
    tf1.compat.v1.keras.backend.set_session(session)
    session.run(tf1.global_variables_initializer())  
    session.run(tf1.tables_initializer())
    # split our data into train and test dataset
    history = model_elmo.fit(x_train, y_train, epochs=2, batch_size=16)
    model_elmo.save_weights('./response-elmo-model.h5')

Train on 4000 samples
Epoch 1/2
4000/4000 [==============================] - 755s 189ms/sample - loss: 0.4022 - acc: 0.8560
Epoch 2/2
4000/4000 [==============================] - 810s 202ms/sample - loss: 0.2740 - acc: 0.9038


Run the test set.

In [ ]:
with tf1.Session() as session:
    tf1.compat.v1.keras.backend.set_session(session)
    session.run(tf1.global_variables_initializer())  
    session.run(tf1.tables_initializer())
    model_elmo.load_weights('./response-elmo-model.h5')  
    predicts  = model_elmo.predict(x_test, batch_size=16)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


In [ ]:
predicts = np.argmax(predicts, axis=1)
rounded_test=np.argmax(y_test, axis=1)
print(metrics.confusion_matrix(rounded_test, predicts))

[[208  14  13   7]
 [  1 206   0   0]
 [  5   0 208  27]
 [  3   1  18 289]]


In [ ]:
print(metrics.classification_report(rounded_test, predicts))

              precision    recall  f1-score   support

           0       0.96      0.86      0.91       242
           1       0.93      1.00      0.96       207
           2       0.87      0.87      0.87       240
           3       0.89      0.93      0.91       311

    accuracy                           0.91      1000
   macro avg       0.91      0.91      0.91      1000
weighted avg       0.91      0.91      0.91      1000

